__Import dependencies and decleare global variables__

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.enable_eager_execution()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

import numpy as np
import pandas as pd
import json
import re
import sys
import os
import time

SEQUENCES_LENGTH= 100
NUM_GENERATE= 1000
EPOCHS = 5
BATCH_SIZE = 64
EMBEDDING_DIM = 256
RNN_DIM = 1024 

__Import western film plots data__

In [2]:
def clean(text):
    '''
    '''
    text = text.lower()
    text = text.replace("ain't", "am not")
    text = text.replace("aren't", "are not")
    text = text.replace("can't", "cannot")
    text = text.replace("can't've", "cannot have")
    text = text.replace("'cause", "because")
    text = text.replace("could've", "could have")
    text = text.replace("couldn't", "could not")
    text = text.replace("couldn't've", "could not have")
    text = text.replace("should've", "should have")
    text = text.replace("should't", "should not")
    text = text.replace("should't've", "should not have")
    text = text.replace("would've", "would have")
    text = text.replace("would't", "would not")
    text = text.replace("would't've", "would not have")
    text = text.replace("didn't", "did not")
    text = text.replace("doesn't", "does not")
    text = text.replace("don't", "do not")
    text = text.replace("hadn't", "had not")
    text = text.replace("hadn't've", "had not have")
    text = text.replace("hasn't", "has not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd", "he would")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd've", "he would have")
    text = text.replace("'s", "")
    text = text.replace("'t", "")
    text = text.replace("'ve", "")
    text = text.replace(".", " . ")
    text = text.replace("!", " ! ")
    text = text.replace("?", " ? ")
    text = text.replace(";", " ; ")
    text = text.replace(":", " : ")
    text = text.replace(",", " , ")
    text = text.replace("´", "")
    text = text.replace("‘", "")
    text = text.replace("’", "")
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("\'", "")
    text = text.replace("\"", "")
    text = text.replace("-", "")
    text = text.replace("–", "")
    text = text.replace("—", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace("{","")
    text = text.replace("}", "")
    text = text.replace("/", "")
    text = text.replace("|", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("$", "")
    text = text.replace("+", "")
    text = text.replace("*", "")
    text = text.replace("%", "")
    text = text.replace("#", "")
    text = text.replace("\r", "")
    text = text.replace("\xa0", "")
    text = text.replace("\u200c", "")
    text = text.replace("\u200d", "")
    text = ''.join([i for i in text if not i.isdigit()])

    return text

try:
    dirname = os.path.abspath('')
    filepath = os.path.join(dirname, 'input_data/wiki_movie_plots.csv')
    dataframe = pd.read_csv(filepath, sep=',')
    plotsList = dataframe['Plot'][dataframe['Genre'] == 'western'].values
    cleanedPlotsList = []
    plotsText = ''
    print('{} plots imported.'.format(len(plotsList)))
    for idx, p in enumerate(plotsList):
        cleaned = clean(p)
        cleanedPlotsList.append(cleaned)
        plotsText += ' ' + cleaned + '\n'
    print('{} plots cleaned.'.format(len(cleanedPlotsList)))
    '''
    path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
    plotsText = open(path_to_file, 'rb').read().decode(encoding='utf-8')
    print ('Length of text: {} characters'.format(len(plotsText)))
    '''
    
except IOError:
    sys.exit('Cannot find data!')


865 plots imported.
865 plots cleaned.


__Extract Vocabulary__

In [3]:
vocabulary = sorted(set(plotsText))
print(vocabulary)
vocab_size = len(vocabulary)
print ('{} unique characters'.format(len(vocabulary)))

char2idx = {u:i for i, u in enumerate(vocabulary)}
idx2char = np.array(vocabulary)
textAsInt = np.array([char2idx[c] for c in plotsText])
print ('{} ---- characters mapped to int ---- > {}'.format(repr(plotsText[:10]), textAsInt[:10]))

['\n', ' ', '!', '&', ',', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'é', 'í', 'ñ', 'ó', 'ü', 'ō', 'ʃ', 'ˈ']
44 unique characters
' the film ' ---- characters mapped to int ---- > [ 1 28 16 13  1 14 17 20 21  1]


## Preprocess

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?
We're going to divide the text into example sequences. Each input sequence will contain SEQUENCES_LENGTH characters from the text. For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of SEQUENCES_LENGTH+1. For example, say SEQUENCES_LENGTH is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [4]:
def split_input_target(chunk):
    inputText = chunk[:-1]
    targetText = chunk[1:]
    return inputText, targetText

# Create training examples and targets
examplesPerEpoch = len(plotsText) // SEQUENCES_LENGTH
stepsPerEpoch = examplesPerEpoch // BATCH_SIZE
print('Steps per Epoch: {}'.format(stepsPerEpoch))

charDataset = tf.data.Dataset.from_tensor_slices(textAsInt)
for i in charDataset.take(5):
    print(idx2char[i.numpy()])
    
sequences = charDataset.batch(SEQUENCES_LENGTH+1, drop_remainder=True)#The batch method lets us easily convert these individual characters to sequences of the desired size.
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
        print("Step {:4d}".format(i))
        print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
        print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

dataset = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True)
dataset

Steps per Epoch: 233
Instructions for updating:
Colocations handled automatically by placer.
 
t
h
e
 
' the film opens with two bandits breaking into a railroad telegraph office ,  where they force the op'
'erator at gunpoint to have a train stopped and to transmit orders for the engineer to fill the locomo'
'tive tender at the station water tank .  they then knock the operator out and tie him up .  as the tr'
'ain stops it is boarded by the banditsnow four .  two bandits enter an express car ,  kill a messenge'
'r and open a box of valuables with dynamite ;  the others kill the fireman and force the engineer to '
Input data:  ' the film opens with two bandits breaking into a railroad telegraph office ,  where they force the o'
Target data: 'the film opens with two bandits breaking into a railroad telegraph office ,  where they force the op'
Step    0
  input: 1 (' ')
  expected output: 28 ('t')
Step    1
  input: 28 ('t')
  expected output: 16 ('h')
Step    2
  input: 16 ('h')
  expe

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

__Build the model__

In [5]:
rnn = tf.keras.layers.CuDNNLSTM 

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
            return_sequences=True,
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

trainModel = build_model(
  vocab_size = vocab_size,
  embedding_dim=EMBEDDING_DIM,
  rnn_units=RNN_DIM,
  batch_size=BATCH_SIZE)

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = trainModel(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

trainModel.summary()

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

(64, 100, 44) # (batch_size, sequence_length, vocab_size)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           11264     
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (64, None, 1024)          5251072   
_________________________________________________________________
dense (Dense)                (64, None, 44)            45100     
Total params: 5,307,436
Trainable params: 5,307,436
Non-trainable params: 0
_________________________________________________________________
Input: 
 'sident ,  a movement that eventually will lead to statehood . \n corset company owner and independent'

Next Char Predictions: 
 'í!aoxˈósg?íjm;s!j h,,ñu!é:fmá?ü;u;íslézmí\ncmtg áíujsí:wzqywik háfy;ló,ólbóñz\nó añhōígquóü:hüñóōaevfr'


__Train the model__

In [6]:
checkpoint_dir = './models/char_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

, callbacks=[checkpoint_callback]

'callbacks=[checkpoint_callback]'

In [7]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

trainModel.compile(
      optimizer = tf.train.AdamOptimizer(),
      loss = loss)

trainModel.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=stepsPerEpoch)

dirname = os.path.abspath('')
weightsPath = os.path.join(dirname, 'models/rnn_char_fables.h5')
trainModel.save_weights(weightsPath)

Epoch 1/5
233/233 [==============================] - 15s 65ms/step - loss: 2.2871
Epoch 2/5
233/233 [==============================] - 14s 59ms/step - loss: 1.7575
Epoch 3/5
233/233 [==============================] - 14s 59ms/step - loss: 1.5178
Epoch 4/5
233/233 [==============================] - 14s 59ms/step - loss: 1.3899
Epoch 5/5
233/233 [==============================] - 14s 59ms/step - loss: 1.3110


__Define generation model__

In [9]:
rnn = tf.keras.layers.CuDNNLSTM

genModel = build_model(
  vocab_size = vocab_size,
  embedding_dim=EMBEDDING_DIM,
  rnn_units=RNN_DIM,
  batch_size=1)

dirname = os.path.abspath('')
weightsPath = os.path.join(dirname, 'models/rnn_char_fables.h5')
genModel.load_weights(weightsPath)
genModel.build(tf.TensorShape([1, None]))
genModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            11264     
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (1, None, 1024)           5251072   
_________________________________________________________________
dense_2 (Dense)              (1, None, 44)             45100     
Total params: 5,307,436
Trainable params: 5,307,436
Non-trainable params: 0
_________________________________________________________________


__Generate text__

In [10]:
def generate_text(model, start_string, char_2_idx, idx_2_char):
    '''
    '''
    # Evaluation step (generating text using the learned weights)
    # Number of characters to generate
    numGenerate = NUM_GENERATE
    # Converting our start string to numbers (vectorizing)
    start_string = clean(start_string) 
    inputEval = [char_2_idx[s] for s in start_string]
    inputEval = tf.expand_dims(inputEval, 0)
    # Empty string to store our results
    textGenerated = []
    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0
    # Here batch size == 1
    model.reset_states()
    
    for i in range(numGenerate):
        predictions = model(inputEval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # using a multinomial distribution to predict the word returned by the trainModel
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
        # We pass the predicted word as the next input to the trainModel
        # along with the previous hidden state
        inputEval = tf.expand_dims([predicted_id], 0)
        textGenerated.append(idx_2_char[predicted_id])

    return (start_string + ''.join(textGenerated))

generated = generate_text(
        model=genModel, 
        start_string="One day three people embark togheter", 
        char_2_idx=char2idx, 
        idx_2_char=idx2char
    )

print(generated)
session.close()

Instructions for updating:
Use tf.random.categorical instead.
one day three people embark togheter offord and that clennett gunages the speaf whombergs to the anchor town of confed ,  grave witnessen ,  send fas and helpest gant remats with clay playsoon  himself that arms of killing a cofft a bar . losing the town torne whom west .  after be syormeever ,  that chearing .  when rybers feel that sherly ranger also works off with the reno when the original office in tadds out hopo james for revenge ,  and eas .  they are can visits cleng of had . 
 by chary tells the stody then jail they escape meets a gunslinge and discovers the outlaws at frenchis after his striger frank and frog olthon instian after the longrass that everyone foll sichurceseman learn bown and kills sheriff when her sky foley after a land . 
the drag its northewer ,  a first turns ,  jonto wanted ,  lile ,  old chivain children and lenc young unamre to paul capture .  corr ,  believes !  locater ,  dispiterces slown jo